# Data drift dashboard in jupyter notebook

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from sklearn import datasets, ensemble

from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab, NumTargetDriftTab, RegressionPerformanceTab

from evidently.model_profile import Profile
from evidently.profile_sections import DataDriftProfileSection

## Bicycle Demand Data

In [3]:
raw_data = pd.read_csv('./train.csv', header=0, sep=',', parse_dates=['datetime'], index_col='datetime')

In [4]:
raw_data.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


## Regression Model

### Feature engineering

In [ ]:
raw_data['month'] = raw_data.index.map(lambda x : x.month)
raw_data['hour'] = raw_data.index.map(lambda x : x.hour)
raw_data['weekday'] = raw_data.index.map(lambda x : x.weekday() + 1)

In [ ]:
raw_data.head()

### Model training

In [ ]:
target = 'count'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'humidity', 'windspeed', 'hour', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
production = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
reference.head()

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
prod_prediction = regressor.predict(production[numerical_features + categorical_features])

In [ ]:
reference['prediction'] = ref_prediction
production['prediction'] = prod_prediction

### Model Perfomance 

In [ ]:
column_mapping = {}

column_mapping['target'] = target
column_mapping['prediction'] = prediction
column_mapping['numerical_features'] = numerical_features
column_mapping['categorical_features'] = categorical_features

In [ ]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab])
regression_perfomance_dashboard.calculate(reference, None, column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
#regression_perfomance_dashboard.save('regression_performance_at_training.html')

##  Week 1

In [ ]:
regression_perfomance_dashboard.calculate(reference, production.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                            column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
#regression_perfomance_dashboard.save('regression_performance_after_week1.html')

## Week 2

In [ ]:
regression_perfomance_dashboard.calculate(reference, production.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
#regression_perfomance_dashboard.save('regression_performance_after_week2.html')

In [ ]:
target_drift_dashboard = Dashboard(tabs=[NumTargetDriftTab])
target_drift_dashboard.calculate(reference, production.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
target_drift_dashboard.show()

In [ ]:
#target_drift_dashboard.save('target_drift_after_week2.html')

## Week 3

In [ ]:
regression_perfomance_dashboard.calculate(reference, production.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
#regression_perfomance_dashboard.save('regression_performance_after_week3.html')

In [ ]:
target_drift_dashboard.calculate(reference, production.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
target_drift_dashboard.show()

In [ ]:
#target_drift_dashboard.save('target_drift_after_week3.html')

## Data Drift

In [ ]:
column_mapping = {}

column_mapping['numerical_features'] = numerical_features

In [ ]:
data_drift_dashboard = Dashboard(tabs=[DataDriftTab])
data_drift_dashboard.calculate(reference, production.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
data_drift_dashboard.show()

In [ ]:
#data_drift_dashboard.save("data_drift_dashboard_after_week1.html")

## Data Drift Profile

In [ ]:
data_drift_profile = Profile(sections=[DataDriftProfileSection])
data_drift_profile.calculate(reference, production.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
data_drift_profile.json()